In [6]:
from sklearn.svm import SVC
import numpy as np
import pickle
import gzip
import pandas as pd
import time
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(font_scale=1.2)
from datetime import datetime
# Allows charts to appear in the notebook
%matplotlib inline

In [7]:
def read_mnist(mnist_file):
    """
    Reads MNIST data.
    
    Parameters
    ----------
    mnist_file : string
        The name of the MNIST file (e.g., 'mnist.pkl.gz').
    
    Returns
    -------
    (train_X, train_Y, val_X, val_Y, test_X, test_Y) : tuple
        train_X : numpy array, shape (N=50000, d=784)
            Input vectors of the training set.
        train_Y: numpy array, shape (N=50000)
            Outputs of the training set.
        val_X : numpy array, shape (N=10000, d=784)
            Input vectors of the validation set.
        val_Y: numpy array, shape (N=10000)
            Outputs of the validation set.
        test_X : numpy array, shape (N=10000, d=784)
            Input vectors of the test set.
        test_Y: numpy array, shape (N=10000)
            Outputs of the test set.
    """
    f = gzip.open(mnist_file, 'rb')
    train_data, val_data, test_data = pickle.load(f, encoding='latin1')
    f.close()
    
    train_X, train_Y = train_data
    val_X, val_Y = val_data
    test_X, test_Y = test_data    
    
    return train_X, train_Y, val_X, val_Y, test_X, test_Y

In [8]:
# Test
train_X, train_Y, val_X, val_Y, test_X, test_Y = read_mnist('mnist.pkl.gz')

print('train_X.shape =', train_X.shape)
print('train_Y.shape =', train_Y.shape)
print('val_X.shape   =', val_X.shape)
print('val_Y.shape   =', val_Y.shape)
print('test_X.shape  =', test_X.shape)
print('test_Y.shape  =', test_Y.shape)

print('\ntrain_X: min = %.3f, max = %.3f' %(train_X.min(), train_X.max()))
print('train_Y: min = %d, max = %d' %(train_Y.min(), train_Y.max()))

train_X.shape = (50000, 784)
train_Y.shape = (50000,)
val_X.shape   = (10000, 784)
val_Y.shape   = (10000,)
test_X.shape  = (10000, 784)
test_Y.shape  = (10000,)

train_X: min = 0.000, max = 0.996
train_Y: min = 0, max = 9


In [9]:
# save the model to disk
def save_model(model, mode,  C_k, gamma_k = 1):
    if mode == 'linear':
        file_name = mode + '_C' + str(C_k) + '.sav'
        pickle.dump(model, open(file_name, 'wb'))
    else:
        file_name = mode + '_C' + str(C_k) + '_gamma' + str(gamma_k) + '.sav'
        pickle.dump(model, open(file_name, 'wb'))
        
# load the model from disk
def load_model(file_name):
    loaded_model = pickle.load(open(file_name, 'rb'))
    return loaded_model

In [10]:
for k in range(-3,4):
    t = 10**k
    s = 10**(-k)
    #Training:
    start_time = datetime.now()
    svclassifier = SVC(C = t,gamma = s,kernel='rbf')
    svclassifier.fit(train_X, train_Y)
    train_time = datetime.now() - start_time
    
    #Save model:
    save_model(svclassifier, 'Gaussian', k)
    
    #Get Y predict from input X
    y_train_pred = svclassifier.predict(train_X)  
    y_val_pred = svclassifier.predict(val_X)
    
    #Get error:
    train_error = 1 -  (accuracy_score(train_Y, y_train_pred, normalize=False) / float(train_Y.size))
    val_error = 1 -  (accuracy_score(val_Y, y_val_pred, normalize=False) / float(val_Y.size))
    
    #Print results:
    print("C: {}  |  gamma: {}  |  train error: {}  |  validation error: {}  |  time: {}".format(t, s, train_error, val_error, train_time))

C: 0.001  |  gamma: 1000  |  train error: 0.88644  |  validation error: 0.8936  |  time: 1:09:56.847450
C: 0.01  |  gamma: 100  |  train error: 0.88644  |  validation error: 0.8936  |  time: 1:32:50.463953
C: 0.1  |  gamma: 10  |  train error: 0.88644  |  validation error: 0.8936  |  time: 1:53:25.034372
C: 1  |  gamma: 1  |  train error: 0.0  |  validation error: 0.8176  |  time: 2:41:24.677824
C: 10  |  gamma: 0.1  |  train error: 0.0  |  validation error: 0.043399999999999994  |  time: 1:33:14.174163
C: 100  |  gamma: 0.01  |  train error: 0.0  |  validation error: 0.016800000000000037  |  time: 0:03:43.154318
C: 1000  |  gamma: 0.001  |  train error: 7.999999999996898e-05  |  validation error: 0.02849999999999997  |  time: 0:03:07.043143


In [ ]:
list_error = []
C_2d_range = [1e-2, 1, 1e2]
gamma_2d_range = [1e-1, 1, 1e1]
for C in C_2d_range:
    for gamma in gamma_2d_range:
        start_time = time.perf_counter()
        svclassifier = SVC(C=C, gamma=gamma,kernel='rbf')
        svclassifier.fit(train_X, train_Y)
        train_time = time.perf_counter() - start_time
        y_pred = svclassifier.predict(test_X)
        my_accuracy = accuracy_score(test_Y, y_pred, normalize=False) / float(test_Y.size)
        error = 1 - my_accuracy
        print("C: {}, gamma: {}, error: {}, time: {}".format(C, gamma, error, train_time))
        list_error.append((error, C, gamma, svclassifier))

C: 0.01, gamma: 0.1, error: 0.7861, time: 3626.6978505


In [8]:
y_pred = svclassifier.predict(test_X)
print(confusion_matrix(test_Y, y_pred))
print(classification_report(test_Y, y_pred))

[[ 974    0    1    0    0    1    1    1    2    0]
 [   0 1130    2    0    0    1    0    1    1    0]
 [   4    1 1015    0    1    0    1    7    3    0]
 [   0    0    1  995    0    3    0    3    5    3]
 [   0    0    3    0  968    0    2    0    0    9]
 [   2    0    0    9    1  871    4    0    3    2]
 [   4    2    1    0    2    4  944    0    1    0]
 [   0    6    9    2    0    0    0 1004    0    7]
 [   3    0    2    4    4    2    1    2  953    3]
 [   3    4    0    6    7    2    0    4    1  982]]
              precision    recall  f1-score   support

           0       0.98      0.99      0.99       980
           1       0.99      1.00      0.99      1135
           2       0.98      0.98      0.98      1032
           3       0.98      0.99      0.98      1010
           4       0.98      0.99      0.99       982
           5       0.99      0.98      0.98       892
           6       0.99      0.99      0.99       958
           7       0.98      0.98   